In [1]:
import glob

import cv2
import matplotlib.pyplot as plt
import numpy as np

from skimage.morphology import skeletonize
from skimage.measure import label, regionprops

In [13]:
# Ineffective!

def detect_fiducials(image_path):
    # 1. Read and convert to grayscale
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 2. Threshold to isolate bright regions (tune 'thresh_value' if needed)
    thresh_value = 200
    _, thresh = cv2.threshold(gray, thresh_value, 255, cv2.THRESH_BINARY)

    # 3. Morphological cleanup to remove small specks and fill gaps
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel)

    # 4. Find contours
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Keep track of potential cross centers
    cross_centers = []

    # 5. Filter and visualize
    for cnt in contours:
        area = cv2.contourArea(cnt)
        # Skip contours that are too small or too large to be crosses
        if area < 50 or area > 20000:
            continue

        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = float(w)/float(h)
        
        # A simple heuristic: crosses are often roughly square in their bounding box
        if 0.8 < aspect_ratio < 1.2:
            # Compute center for alignment or further processing
            cx, cy = x + w//2, y + h//2
            cross_centers.append((cx, cy))

            # Draw a rectangle around the contour on the original image
            print(f"detected: {(x, y)}")
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # 6. Show results side by side
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.title('Thresholded Image')
    plt.imshow(thresh, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title('Detected Fiducials')
    # Convert from BGR to RGB for correct color display with matplotlib
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    return cross_centers

In [18]:
def non_max_suppression(locations, radius=10):
    """
    Given a list of (x, y, match_score) tuples and a suppression radius,
    return a filtered list so that no two points are closer than 'radius'.
    """
    if not locations:
        return []

    # Sort by score descending so we keep strongest matches first
    locations = sorted(locations, key=lambda x: x[2], reverse=True)
    kept = []

    for (x, y, score) in locations:
        too_close = False
        for (kx, ky, kscore) in kept:
            # Euclidean distance check
            if (x - kx)**2 + (y - ky)**2 < radius**2:
                too_close = True
                break
        if not too_close:
            kept.append((x, y, score))

    return kept

def detect_crosses_via_template(chip_path, template_path, match_thresh=0.8):
    """
    Detect plus-shaped fiducials in 'chip_path' image by matching
    a known 'template_path' plus shape.
    """
    # 1. Read images (grayscale)
    chip_img = cv2.imread(chip_path, cv2.IMREAD_GRAYSCALE)
    template_img = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)

    # Check that they loaded
    if chip_img is None or template_img is None:
        raise IOError("Could not read chip or template image!")

    # 2. Optionally threshold or preprocess if desired
    #    (Sometimes it helps to invert the template or the main image.)
    # For demonstration, let's do a slight blur on the chip image
    chip_blur = cv2.GaussianBlur(chip_img, (3, 3), 0)

    # 3. MatchTemplate
    # TM_CCOEFF_NORMED or TM_CCORR_NORMED are commonly used methods.
    res = cv2.matchTemplate(chip_blur, template_img, cv2.TM_CCOEFF_NORMED)

    # 4. Threshold the match score
    # This creates a mask for all points above match_thresh
    match_mask = (res >= match_thresh)
    ys, xs = np.where(match_mask)

    # Collect matches with their correlation score
    # We store the *center* of the matched region
    h_t, w_t = template_img.shape[:2]
    match_points = []
    for (x, y) in zip(xs, ys):
        score = res[y, x]
        center_x = x + w_t // 2
        center_y = y + h_t // 2
        match_points.append((center_x, center_y, score))

    # 5. Non-maximum suppression to reduce duplicates
    filtered = non_max_suppression(match_points, radius=10)

    # 6. Visualize results
    # Convert to color for drawing
    chip_color = cv2.cvtColor(chip_img, cv2.COLOR_GRAY2BGR)
    for (cx, cy, sc) in filtered:
        print(f"detected: {cx, cy}")
        cv2.drawMarker(
            chip_color, (cx, cy), (0, 0, 255),
            markerType=cv2.MARKER_CROSS, markerSize=20, thickness=2
        )

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.title("Template Image")
    plt.imshow(template_img, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("Detected Crosses (red markers)")
    plt.imshow(chip_color[..., ::-1])  # BGR to RGB
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    # Print or return the final coordinates
    final_coords = [(int(cx), int(cy)) for (cx, cy, sc) in filtered]
    return final_coords

In [2]:
def detect_crosses_skeleton(image_path):
    # 1) Read & Threshold
    img_bgr = cv2.imread(image_path)
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # Adjust threshold as needed
    _, bw = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

    # 2) Morphological open/close
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    bw = cv2.morphologyEx(bw, cv2.MORPH_OPEN, kernel)
    bw = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)

    # 3) Skeletonize (skimage wants bool or 0/1)
    bw_bool = (bw > 0).astype(np.uint8)
    skeleton = skeletonize(bw_bool).astype(np.uint8)

    # 4) Identify cross candidates
    # We'll look for "branch points" in the skeleton. A branch point
    # is a pixel with 3+ neighbors that are also skeleton pixels.
    # For a perfect cross, we want exactly 4 neighbors.
    crosses = []
    height, width = skeleton.shape
    for y in range(1, height-1):
        for x in range(1, width-1):
            if skeleton[y, x] == 1:
                # Count how many 8-connected neighbors are also 1
                # (You might do 4-connected if your cross is purely orth.)
                neighbors = 0
                coords = []
                for ny in range(y-1, y+2):
                    for nx in range(x-1, x+2):
                        if (ny, nx) != (y, x) and skeleton[ny, nx] == 1:
                            neighbors += 1
                            coords.append((nx, ny))

                # If we want exactly 4 arms in an orthogonal cross,
                # we might prefer 4-connected neighbors. 
                # But let's proceed with 8-connected logic to find "branch points."
                if neighbors == 4:
                    # We found a candidate center of a cross. 
                    # Optionally check orientation of each neighbor
                    # or measure arm lengths in each direction, etc.
                    crosses.append((x, y))

    # We'll mark these in the color image
    for (cx, cy) in crosses:
        print(f"detected: {cx, cy}")
        cv2.drawMarker(img_bgr, (cx, cy), (0,0,255), markerType=cv2.MARKER_CROSS, markerSize=15, thickness=2)

    # Show
    fig, axes = plt.subplots(1,2, figsize=(12,6))
    axes[0].imshow(skeleton, cmap='gray')
    axes[0].set_title("Skeleton")
    axes[0].axis('off')

    axes[1].imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
    axes[1].set_title("Detected Cross Centers")
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

    return crosses


In [3]:
images = glob.glob("litho_captures/*.png")